<a href="https://colab.research.google.com/github/batsoyla/E6.5_2016-17/blob/master/Bert_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Πηγαίνω στον φάκελο που έχω αποθηκεύσει τον φάκελο File_sent_analysis

In [0]:
%cd/content/drive/My Drive/MyThesis/File_sent_analysis

/content/drive/My Drive/MyThesis/File_sent_analysis


In [0]:
!pip uninstall tensorflow
!pip install tensorflow==1.15.0

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0
     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 512kB 50.9MB/s 
     |████████████████████████████████| 3.8MB 47.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=b98dfc6a2a341ca690b0f502e605287be9caed574b00a543c79e248ab7248eb1
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have 

# **ΑΡΧΙΚΟ ΔΕΙΓΜΑ ΔΕΔΟΜΕΝΩΝ**

Αρχικά εντοπίζεται το path και φορτώνεται ολόκληρο το δείγμα δεδομένων, διαγραφονται οι στήλες του πίνακα, μετονομάζονται κατάλληλα και γίνονται κάποιες αρχικές οπτικοποιήσεις, για να έχουμε μια πρώτη πολύ πρώιμη εικόνα των δεδομένων. Τα δεδομένα αρχικά βρίσκονται στον φάκελο Crawled_Data και είναι σε μορφή Json αρχείων.Επιλέχθηκε να πραγματοποιηθεί binary sentiment analysis και ετσι όσες κριτικές έλαβαν 4 ή 5 αστεράκια να θεωρηθούν θετικές και όσες έλαβαν 2 ή 1 να θεωρηθούν αρνητικές και να μην χρησιμοποιηθούν καθόλου ουδέτερες κριτικές.

In [0]:
import os 
import pandas as pd
#from Data_Process import dictionaries
import dictionaries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tabulate import _table_formats, tabulate


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
#merge the given datafiles
def merge_to_dataframe_save(path):
  list_of_dataframes=[]
  list_crawled_data_files=os.listdir(path)
  a=os.path.join(path, list_crawled_data_files[0])
#erase file if it exists:
  for item in list_crawled_data_files:
    if os.path.isfile(os.path.join(path, item)):
      if os.path.join(path, item)[:65] != "/content/drive/My Drive/MyThesis/Thesis/Crawled_Data/TripAdvisor_":
        os.remove(os.path.join(path, item))
      else:
        df_temp=pd.read_json(os.path.join(path, item), lines = True)
        list_of_dataframes.append(df_temp)
  df=pd.concat(list_of_dataframes,  ignore_index=True)
  df.to_json(os.path.join(path, "Crawled_Data.json"), orient="records", lines=True)
  return df

#create polarity column
def pol_column(value):
  if value > 3:
    return 1
  elif value < 3:
    return 0

#create sentiment column
def sentiment_column(n):
  switcher = {
      1: "Positive",
      0: "Negative" 
    }
  return switcher.get(n, "Invalid data")

#flatten list of lists
def flatten(list_of_lists):
  for sublist in list_of_lists:
    string="".join(sublist)
  return string

def sent_pol_table(data):
  table = [["Negative", 0, len(data[data["Sentiment"] =="Negative"])],["Positive", 1, len(data[data["Sentiment"] =="Positive"])]]
  headers = ["Sentiment", "Polarity","No_rows"]
  print(tabulate(table, headers, tablefmt="psql"))


In [0]:
#path retrieval
crawled_data_path = os.path.abspath("Crawled_Data")

# merge dataframe
df=merge_to_dataframe_save(crawled_data_path)

#create polarity column
df["Polarity"]=df["Star"].apply(pol_column)

# create sentiment column
df["Sentiment"]=df["Polarity"].apply(sentiment_column)

#drop columns that we don't need
df = df.drop(["Review_title", "Rating_Date", "Restaurant_Name", "Star"], axis=1)

#flatten list of lists
df["Review"]=df["Review"].apply(flatten)

#Drop Null/NA Values from DataFrame
df=df.dropna()
df=df.reset_index(drop=True)

#drop duplicate rows
df.drop_duplicates(inplace=True)
df=df.reset_index(drop=True)

#print occurencies per class
sent_pol_table(df)
sns.countplot(x="Polarity", data=df, palette="husl")
plt.xlabel("Review Score")
plt.title("Unbalanced Dataset")


IndexError: ignored

# **ΠΡΟΕΠΕΞΕΡΓΑΣΙΑ ΔΕΔΟΜΕΝΩΝ**
***ΔΗΜΙΟΥΡΓΙΑ ΙΣΟΡΡΟΠΗΜΕΝΟΥ ΔΕΙΓΜΑΤΟΣ***

---




Από τις παραπάνω οπτικοποιήσεις γίνεται προφανές ότι το δείγμα δεδομένων παρουσιάζει αισθητή ανισορροπία κάτι που δυνητικά επηρεάζει την εκπάιδευση του μοντέλου, το μοντέλο πολύ πιθανόν να "προβλέπει" την συχνότερη κλάση χωρίς την ανάλογη ανάλυση δίνοντας έτσι διαστρευλωμένα αποτελέσματα ακρίβειας πρόβλεψης. Για την εξισορρόπηση του δείγματος επιλέγεται η τεχνική του *undersampling* όπου απο ένα πολύ μεγαλύτερο δείγμα επιλέγεται να αφαιρεθούν γραμμές από τις επικρατούσες κλάσεις, οδηγώντας έτσι σε ένα μικρότερο αλλά ισορροπιμένο δείγμα συνολικά **print(df.shape[0])** γραμμών.

In [0]:
#occurencies of min class
min_occ=df.groupby("Polarity")["Sentiment"].count().min()

#shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)
target_count = df.Sentiment.value_counts()

# devide dataset by class
df_pos = df[df["Polarity"] == 1]
df_neg = df[df["Polarity"] == 0]

# Random under-sampling
df_pos = df_pos.sample(min_occ)
df_neg = df_neg.sample(min_occ)

#concatenate to new dataset re-shuffle and re-index data
df_under = pd.concat([df_pos, df_neg], axis=0).sample(frac=1)

#we will keep the rest of the dataframe to make predictions (left_join)
df_pred = df.merge(df_under, on=["Review"], how='left', indicator=True)
df_pred = df_pred[df_pred["_merge"] == "left_only"]
df_pred = df_pred.drop(["_merge","Polarity_y", "Sentiment_y"], axis=1)
df_pred =df_pred.reset_index(drop=True)
df_pred.rename(columns={"Polarity_x": "Polarity", "Sentiment_x":"Sentiment"}, inplace=True)
df_under=df_under.reset_index(drop=True)

#save the final dataset
df_under.to_json(os.path.join(crawled_data_path, 'Data_to_Process.json'), orient='records', lines=True)

#print new dataset
sent_pol_table(df_under)
sns.countplot(x="Sentiment", data=df_under, palette="husl")
plt.xlabel("Review Score")
plt.title("New Balanced Dataset")

# ***ΚΑΘΑΡΙΣΜΟΣ ΔΕΙΓΜΑΤΟΣ*** 


---
Αρχικά επιλέχθηκέ να γίνει μια προεπεξεργασία του δείγματος που στόχο έχει την διάνθισει της πληροφορία του δείγματος και την ορθότερη απόδοση του, χωρίς ορθογραφικά λάθη και άχρηστες πληροφορίες που μόνο "θόρυβο" προκαλούν στο δείγμα μας.
Για τον καθαρισμό των δεδομένων μας επιλέχθηκαν απλές τεχνικές όπως

1.   Εξάληψη επιπλέον κενών 
2.   Απόδοση σύντομεύσεων σε πλήρες κείμενο 
3. Αντικάτάσταση emoji/emoticons με αντιπροσωπευτηκές λέξης
4. Διαγραφή URL
5. Ορθογραφικός έλεγχος 


In [0]:
import re

In [0]:
#remove url
df_under["Review"]=df_under["Review"].str.replace('http\S+|www.\S+', '', case=False)

#lowercase and remove extra blanks,tabs etc
df_under["Review"]=df_under["Review"].apply(lambda x: " ".join(x.lower() for x in x.split()))

# replace repeated characers (only when character is repeated more than two times)
df_under["Review"]=df_under["Review"].apply(lambda x: ' '.join(re.sub(r"(.)\1+", r'\1\1', item) for item in x.split()))

#replace emojis/emoticons
df_under["Review"]=df_under["Review"].apply(lambda x: ' '.join(dictionaries.emoticons_emo.get(item) if item  in dictionaries.emoticons_emo else item for item in x.split()))
df_under["Review"]=df_under["Review"].apply(lambda x: ' '.join(dictionaries.emoticons.get(item) if item  in dictionaries.emoticons else item for item in x.split()))

#All the apostrophes to be converted into standard lexicons APPOSTOPHES[word] if word in APPOSTOPHES else word for word in words]
df_under["Review"]=df_under["Review"].apply(lambda x: ' '.join(dictionaries.appos.get(item) if item  in dictionaries.appos  else item for item in x.split()))

#replace slang to actual correct words
df_under["Review"]=df_under["Review"].apply(lambda x: ' '.join(dictionaries.slang.get(item) if item  in dictionaries.slang  else item for item in x.split()))


Γίνεται ορθογραφικός έλεγχος ένα βήμα αρκετά χρονοβόρο καθώς χρειάζεται στο δείγμα μας περίπου ... για να ολοκληρωθεί.

In [0]:
!pip install pyspellchecker
import time

#correct grammar
start_time = time.time()
from spellchecker import SpellChecker
spell = SpellChecker()
df_under['Review']=df_under['Review'].apply(lambda x: ' '.join(spell.correction(item) for item in x.split()))
print("Execution time : %s" % (time.time() - start_time))


In [0]:
df_under.to_json(os.path.join(crawled_data_path, 'Data_to_Process.json'), orient='records', lines=True)


Ακολουθούν κάποιες οπτικοποιήσεις καθώς και η αφαίρεση κάποιων στήλων από τα δεδομένα που πλέον δεν χρειάζονται.

In [0]:
import math
def round_half_up(n, decimals=0):
  multiplier = 10 ** decimals
  return math.floor(n*multiplier + 0.5) / multiplier

In [0]:
from tabulate import _table_formats, tabulate

#calculate mean of words
df_under["words_number"]=df_under["Review"].str.split().str.len()

#calculate number of sentences
df_under["sentences_number"]=df["Review"].str.split(".").str.len()
table = [["Average number of words", round_half_up(df_under["words_number"].mean(),2)],["Anerage number of sentences", round_half_up(df_under["sentences_number"].mean(),2)]]
print(tabulate(table, tablefmt="psql"))

#drop columns
df_under = df_under.drop([ "words_number", "sentences_number"], axis=1)

#change the order of columns
df_under = df_under[["Review","Sentiment","Polarity"]]

# **ΑΝΑΛΥΣΗ ΣΥΝΑΙΣΘΗΜΑΤΟΣ ΜΕ ΧΡΗΣΗ ΤΟΥ ΜΟΝΤΕΛΟΥ BERT**

---
Αρχικά εισάγωνται οι απαραιτητες βιβλιοθήκες.



In [0]:
!pip install bert-tensorflow

In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

#/usr/local/lib/python3.6/dist-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

In [0]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [0]:
print(tf.__version__)

Χωρίζουμε τα δεδομένα μας για την εκπαιδευση και αξιολόγηση του μοντέλου: 80% για την εκπαίδευση και το υπόλοιπο 20% για την αξιολογηση του.

In [0]:
train,test=train_test_split(df_under,test_size=0.2)


In [0]:
DATA_COLUMN = "Review"
LABEL_COLUMN = "Polarity"
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

Πρέπει το δείγμα μας να έρθει στην μορφή που το μοντέλο BERT "καταλαβαίνει". Προς αυτή την κατεύθυνση λοιπόν δημιουργούμε τα train_InputExamples και test_InputExamples χρησιμοποιώντας τον κατασκευαστή που παρέχεται στην βιβλιοθήκη του μοντέλου.


*   text_a = είναι το δείγμα κειμένου που θέλουμε να κατηγοριοποιήσουμε
*   text_b = χρησιμοποιείται όταν στόχος είναι ο προσδιορισμός σχέσεων που υπάρχουν ανάμεσα στις προτάσεις, κάτι που δεν μας αφορά στην περιπτωσή μας άρα και αφήνεται κενό.
*   label = οι κατηγορίες που προσδιορίσαμε είναι οι 0 και 1 που αποδίδουν την αρνητική και θετική κριτική αντίστοιχα.


In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

Το μοντέλο που χρησιμοποιείται είναι το BERT-Base, CASE (12-layer, 768-hidden, 12-heads, 110M). Τα δεδομένα μας τελικά πρέπει να επεξεργαστούν για να έχουν και την μορφή στην οποία έχει εκπαιδευτεί το μοντέλο όπως ήδη αναφέρθηκε παραπάνω. Ετσι ακολουθούν και οι παρακάτω ενέργειες:






*  Μετατροπή σε πεζά.
*   List it Διαχωρισμός στα κομμάτια της πρότασης (tokenization).
*    Διάσπαση λέξεων σε WordPieces (δηλαδή "calling" -> ["call", "##ing"])
*   Αντιστοιχίση λέξεων σε ευρετήρια χρησιμοποιώντας το αρχείο που παρεχει το μοντέλο.
*    Προσθήκη ειδικών αναγνωριστικών "CLS" και "SEP".
*   Προσθήκη "index" και "segment" σε κάθε τιμή εισόδου.*   List item




In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.compat.v1.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()
#WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead

In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

# **ΔΗΜΙΟΥΡΓΙΑ ΜΟΝΤΕΛΟΥ**

---



In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Στη συνέχεια το μοντέλο ολοκληρώνεται με την μέθοδο model_fn_builder που προσαρμόζει το μοντέλο μας ώστε να μπορεί να εκπαιδευτεί να κάνει αξιολογήσεις καθώς και προβλέψεις.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir="/content/drive/My Drive/MyThesis/Thesis/Output_dir",
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Χρησιμοποιούνται εδώ τα δεδομένα που προορίζονται για την αξιολόγηση του μοντέλου.

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

Ακολουθούν κάποιες προβλέψεις:

In [0]:
def getPrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return  [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
print(df_pred["Review"].head(20))

Το μοντέλο έχει εκπαιδευτεί και μπορεί πλέον να κάνει προβλέψεις.Ακολουθούν κάποιες προβλέψεις από το δείγμα που απέμεινε από την εξισορρόπηση του δείγματος.

In [0]:
predict_sentences=df_pred["Review"].head(10).tolist()

In [0]:
predictions_examples=getPrediction(predict_sentences)

In [0]:
df = pd.DataFrame(predictions_examples, columns=['Review','Prediction_Probabilities','Predicted_label'])

In [0]:
inner_join_df= pd.merge(df, df_pred.head(10), on='Review', how='inner')
inner_join_df = inner_join_df.drop(columns=['Prediction_Probabilities', 'Polarity'])

In [0]:
print(inner_join_df)